In [1]:
import pandas as pd
import numpy as np
import glob
import gc
import keras 
from keras.layers import LSTM, Dense, TimeDistributed, Dropout, MaxPooling2D, Conv2D, Input
from keras.layers import Flatten, BatchNormalization, Activation, Reshape, concatenate
from keras.models import Model
import random
from sklearn.model_selection import KFold
import keras.backend as K
from sklearn.utils import class_weight

gc.enable()

Using TensorFlow backend.


In [2]:
spectrums_index = np.array(glob.glob('../preparations/spectrograms/*'))
labels_index = np.array(glob.glob('../preparations/labels/*'))

In [3]:
num_samples = 2000
height = 108
width = 108
steps = 20
spec_list = spectrums_index[:num_samples]
labe_list = labels_index[:num_samples]

In [5]:
kf = KFold(n_splits=3, random_state=7)

In [6]:
def classifier_1(optimizer='adam', loss='binary_crossentropy', metrix='accuracy'):
    #cnn
    cnn_inputs = Input(shape=(height, width, 1), name='cnn_inputs')
    layers = BatchNormalization()(cnn_inputs)
    layers = Conv2D(32, (3,3), padding="same", activation="relu")(layers)
    layers = MaxPooling2D(2,2, border_mode='same')(layers)
    layers = Conv2D(32, (3,3), activation='relu')(layers)
    layers = BatchNormalization()(layers)
    layers = MaxPooling2D(2,2, border_mode='same')(layers)
    layers = Flatten()(layers)
    layers = Dropout(0.3)(layers)
    layers = Dense(32, activation='relu')(layers)
    
    #lstm
    lstm_inputs = Input(shape=(steps, 24), name='lstm_inputs')
    lstm_layers = LSTM(16, return_sequences=True)(lstm_inputs)
    lstm_layers = LSTM(16)(lstm_layers)
    lstm_layers = Dense(24, activation='relu')(lstm_layers)
    
    #main route
    main_ = concatenate([lstm_layers, layers])
    main_ = BatchNormalization()(main_)
    main_ = Dense(64, activation='relu')(main_)
    output = Dense(24, activation='sigmoid')(main_)
    
    model = Model(inputs=[lstm_inputs, cnn_inputs], outputs=[output])
    model.compile(optimizer=optimizer, loss=loss, metrics=[metrix])
    return model

In [7]:
def classifier_3(optimizer='adam', loss='binary_crossentropy', metrix='accuracy'):
    #cnn
    cnn_inputs = Input(shape=(height, width, 1), name='cnn_inputs')
    layers = Conv2D(64, (3,3), padding="same", activation="relu")(layers)
    layers = BatchNormalization()(layers)
    layers = MaxPooling2D(2,2, border_mode='same')(layers)
    layers = Flatten()(layers)
    layers = Dropout(0.3)(layers)
    layers = Dense(32, activation='relu')(layers)
    
    #lstm
    lstm_inputs = Input(shape=(steps, 24), name='lstm_inputs')
    lstm_layers = LSTM(16, return_sequences=True)(lstm_inputs)
    lstm_layers = LSTM(16)(lstm_layers)
    lstm_layers = Dense(24, activation='relu')(lstm_layers)
    
    #main route
    main_ = concatenate([lstm_layers, layers])
    main_ = BatchNormalization()(main_)
    main_ = Dense(64, activation='relu')(main_)
    output = Dense(24, activation='sigmoid')(main_)
    
    model = Model(inputs=[lstm_inputs, cnn_inputs], outputs=[output])
    model.compile(optimizer=optimizer, loss=loss, metrics=[metrix])
    return model

In [8]:
def classifier_4(optimizer='adam', loss='binary_crossentropy', metrix='accuracy'):
    #cnn
    cnn_inputs = Input(shape=(height, width, 1), name='cnn_inputs')
    layers = MaxPooling2D(2,2, border_mode='same')(cnn_inputs)
    layers = Conv2D(32, (3,3), padding="same", activation="relu")(layers)
    layers = BatchNormalization()(layers)
    layers = Conv2D(32, (3,3), activation='relu')(layers)
    layers = MaxPooling2D(2,2, border_mode='same')(layers)
    layers = Conv2D(32, (3,3), activation='relu')(layers)
    layers = Flatten()(layers)
    layers = Dropout(0.3)(layers)
    layers = Dense(32, activation='relu')(layers)
    
    #lstm
    lstm_inputs = Input(shape=(steps, 24), name='lstm_inputs')
    lstm_layers = LSTM(16, return_sequences=True)(lstm_inputs)
    lstm_layers = LSTM(16)(lstm_layers)
    lstm_layers = Dense(24, activation='relu')(lstm_layers)
    
    #main route
    main_ = concatenate([lstm_layers, layers])
    main_ = BatchNormalization()(main_)
    main_ = Dense(64, activation='relu')(main_)
    output = Dense(24, activation='sigmoid')(main_)
    
    model = Model(inputs=[lstm_inputs, cnn_inputs], outputs=[output])
    model.compile(optimizer=optimizer, loss=loss, metrics=[metrix])
    return model

In [9]:
train_order = []
test_order = []
length = np.arange(num_samples)

for x, y in kf.split(range(num_samples)):
    train_order.append(length[x])
    test_order.append(length[y])
train_order = np.array(train_order)
test_order = np.array(test_order)

In [10]:
def inputs_to_model(spectrum_list, labels_list, order, num, start, ending):
    lstm_output = []
    cnn_output = []
    y_train = []
    
    import time
    start_ = time.time()

    for file in order[num][start:ending]:
        spectrum = np.load(spectrum_list[file])
        labels = np.load(labels_list[file])
        
        _lstm = np.zeros((20, 24))
        for count, i in enumerate(labels):
            _cnn = spectrum[:, 27*count:108+27*count]
            if _cnn.shape == (height, width):
                lstm_output.append(_lstm)
                cnn_output.append(_cnn.reshape((height, width, 1)))
                y_train.append(i)
        
            _lstm = _lstm[1:, :]
            _lstm = np.append(_lstm, i.reshape((1, 24)), axis=0)
        
    lstm_inputs = np.array(lstm_output)
    cnn_inputs = np.array(cnn_output)
    labels_inputs = np.array(y_train)
    print(time.time()-start_)
    return lstm_inputs, cnn_inputs, labels_inputs

In [11]:
lstm_test, cnn_test, labels_test = inputs_to_model(spec_list, labe_list, test_order, 0, 0, test_order[0].shape[0]+1)

11.777542352676392


In [12]:
def train_test_whole(classifier, num, name, spectrum_list, labels_list, epochs=100, batchsize=32):
    lstm_inputs, cnn_inputs, labels_inputs = inputs_to_model(spectrum_list, labels_list, train_order, 
                                                             num, 0, train_order[num].shape[0]+1)

    classifier.fit([lstm_inputs, cnn_inputs], labels_inputs, epochs=epochs, batch_size=batchsize, verbose=1,
                  validation_data = ([lstm_test, cnn_test], labels_test))
    
    classifier.save_weights(name, overwrite=True)   
    del classifier
    K.clear_session()

In [121]:
adam = keras.optimizers.Adam(lr=0.005)
classifier = classifier_1(optimizer=adam)
train_test_whole(classifier, 0, 'weights/weight-75overlap-model1.hdf5', spec_list, labe_list, epochs=10)
gc.collect()

c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  if __name__ == '__main__':


1556.4078123569489
Train on 164164 samples, validate on 82075 samples
Epoch 1/50
164164/164164 [==============================] - 299s 2ms/step - loss: 0.0454 - acc: 0.9830 - val_loss: 0.0395 - val_acc: 0.9840
Epoch 2/50
164164/164164 [==============================] - 270s 2ms/step - loss: 0.0323 - acc: 0.9870 - val_loss: 0.0366 - val_acc: 0.9849
Epoch 3/50
164164/164164 [==============================] - 272s 2ms/step - loss: 0.0286 - acc: 0.9883 - val_loss: 0.0304 - val_acc: 0.9882
Epoch 4/50
164164/164164 [==============================] - 271s 2ms/step - loss: 0.0263 - acc: 0.9891 - val_loss: 0.0311 - val_acc: 0.9877
Epoch 5/50
164164/164164 [==============================] - 272s 2ms/step - loss: 0.0247 - acc: 0.9896 - val_loss: 0.0263 - val_acc: 0.9894
Epoch 6/50
164164/164164 [==============================] - 271s 2ms/step - loss: 0.0233 - acc: 0.9902 - val_loss: 0.0263 - val_acc: 0.9893
Epoch 7/50
164164/164164 [==============================] - 275s 2ms/step - loss: 0.0224 -

KeyboardInterrupt: 

In [13]:
adam = keras.optimizers.Adam(lr=0.005)
classifier = classifier_1(optimizer=adam)
train_test_whole(classifier, 0, 'weights/weight-75overlap-model1.hdf5', spec_list, labe_list, epochs=10)
gc.collect()

c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  if __name__ == '__main__':


22.86003351211548
Train on 164164 samples, validate on 82075 samples
Epoch 1/10
164164/164164 [==============================] - 268s 2ms/step - loss: 0.0456 - acc: 0.9830 - val_loss: 0.0416 - val_acc: 0.9850
Epoch 2/10
164164/164164 [==============================] - 264s 2ms/step - loss: 0.0329 - acc: 0.9867 - val_loss: 0.0300 - val_acc: 0.9877
Epoch 3/10
164164/164164 [==============================] - 262s 2ms/step - loss: 0.0295 - acc: 0.9879 - val_loss: 0.0367 - val_acc: 0.9861
Epoch 4/10
164164/164164 [==============================] - 268s 2ms/step - loss: 0.0272 - acc: 0.9888 - val_loss: 0.0266 - val_acc: 0.9888
Epoch 5/10
164164/164164 [==============================] - 287s 2ms/step - loss: 0.0258 - acc: 0.9893 - val_loss: 0.0259 - val_acc: 0.9891
Epoch 6/10
164164/164164 [==============================] - 287s 2ms/step - loss: 0.0247 - acc: 0.9897 - val_loss: 0.0241 - val_acc: 0.9899
Epoch 7/10
164164/164164 [==============================] - 284s 2ms/step - loss: 0.0238 - 

22422

In [16]:
rmsprop = keras.optimizers.RMSprop()
classifier = classifier_1(optimizer=rmsprop)
train_test_whole(classifier, 0, 'weights/weight-75overlap-model1-rmsprop.hdf5', spec_list, labe_list, epochs=10)
gc.collect()

c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  """
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  import sys
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  # Remove the CWD from sys.path while we load stuff.


4.730387210845947
Train on 164164 samples, validate on 82075 samples
Epoch 1/20
164164/164164 [==============================] - 267s 2ms/step - loss: 0.0558 - acc: 0.9802 - val_loss: 0.0388 - val_acc: 0.9845
Epoch 2/20
164164/164164 [==============================] - 264s 2ms/step - loss: 0.0387 - acc: 0.9848 - val_loss: 0.0361 - val_acc: 0.9862
Epoch 3/20
164164/164164 [==============================] - 264s 2ms/step - loss: 0.0359 - acc: 0.9858 - val_loss: 0.0414 - val_acc: 0.9856
Epoch 4/20
164164/164164 [==============================] - 261s 2ms/step - loss: 0.0343 - acc: 0.9866 - val_loss: 0.0359 - val_acc: 0.9851
Epoch 5/20
164164/164164 [==============================] - 262s 2ms/step - loss: 0.0331 - acc: 0.9870 - val_loss: 0.0376 - val_acc: 0.9847
Epoch 6/20
 27744/164164 [====>.........................] - ETA: 3:13 - loss: 0.0326 - acc: 0.9871

KeyboardInterrupt: 

In [123]:
adam = keras.optimizers.Adam(lr=0.005)
classifier = classifier_3(optimizer=adam)
train_test_whole(classifier, 0, 'weights/weight-75overlap-model3.hdf5', spec_list, labe_list, epochs=10)
gc.collect()

c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  after removing the cwd from sys.path.
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  import sys


1304.7607724666595
Train on 164164 samples, validate on 82075 samples
Epoch 1/50
164164/164164 [==============================] - 259s 2ms/step - loss: 0.0486 - acc: 0.9821 - val_loss: 0.0356 - val_acc: 0.9862
Epoch 2/50
164164/164164 [==============================] - 247s 2ms/step - loss: 0.0371 - acc: 0.9853 - val_loss: 0.0346 - val_acc: 0.9862
Epoch 3/50
164164/164164 [==============================] - 250s 2ms/step - loss: 0.0345 - acc: 0.9862 - val_loss: 0.0312 - val_acc: 0.9873
Epoch 4/50
164164/164164 [==============================] - 254s 2ms/step - loss: 0.0331 - acc: 0.9867 - val_loss: 0.0311 - val_acc: 0.9873
Epoch 5/50
164164/164164 [==============================] - 253s 2ms/step - loss: 0.0324 - acc: 0.9870 - val_loss: 0.0321 - val_acc: 0.9873
Epoch 6/50
 73152/164164 [============>.................] - ETA: 2:04 - loss: 0.0315 - acc: 0.9872

KeyboardInterrupt: 

In [17]:
adam = keras.optimizers.Adam(lr=0.005)
classifier = classifier_3(optimizer=adam)
train_test_whole(classifier, 0, 'weights/weight-75overlap-model3.hdf5', spec_list, labe_list, epochs=10)
gc.collect()

c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  after removing the cwd from sys.path.
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  import sys


10.057117938995361
Train on 164164 samples, validate on 82075 samples
Epoch 1/20
164164/164164 [==============================] - 275s 2ms/step - loss: 0.0471 - acc: 0.9826 - val_loss: 0.0352 - val_acc: 0.9863
Epoch 2/20
164164/164164 [==============================] - 265s 2ms/step - loss: 0.0342 - acc: 0.9865 - val_loss: 0.0305 - val_acc: 0.9876
Epoch 3/20
164164/164164 [==============================] - 265s 2ms/step - loss: 0.0312 - acc: 0.9874 - val_loss: 0.0296 - val_acc: 0.9877
Epoch 4/20
164164/164164 [==============================] - 265s 2ms/step - loss: 0.0295 - acc: 0.9881 - val_loss: 0.0300 - val_acc: 0.9881
Epoch 5/20
164164/164164 [==============================] - 270s 2ms/step - loss: 0.0284 - acc: 0.9884 - val_loss: 0.0297 - val_acc: 0.9880
Epoch 6/20
164164/164164 [==============================] - 264s 2ms/step - loss: 0.0276 - acc: 0.9886 - val_loss: 0.0281 - val_acc: 0.9883
Epoch 7/20
164164/164164 [==============================] - 264s 2ms/step - loss: 0.0269 -

KeyboardInterrupt: 

In [22]:
def classifier_3(optimizer='adam', loss='binary_crossentropy', metrix='accuracy'):
    #cnn
    cnn_inputs = Input(shape=(height, width, 1), name='cnn_inputs')
    layers = MaxPooling2D(2,2, border_mode='same')(cnn_inputs)
    layers = Conv2D(256, (3,3), padding="same", activation="relu")(layers)
    layers = BatchNormalization()(layers)
    layers = MaxPooling2D(2,2, border_mode='same')(layers)
    layers = Flatten()(layers)
    layers = Dropout(0.3)(layers)
    layers = Dense(32, activation='relu')(layers)
    
    #lstm
    lstm_inputs = Input(shape=(steps, 24), name='lstm_inputs')
    lstm_layers = LSTM(16, return_sequences=True)(lstm_inputs)
    lstm_layers = LSTM(16)(lstm_layers)
    lstm_layers = Dense(24, activation='relu')(lstm_layers)
    
    #main route
    main_ = concatenate([lstm_layers, layers])
    main_ = BatchNormalization()(main_)
    main_ = Dense(64, activation='relu')(main_)
    output = Dense(24, activation='sigmoid')(main_)
    
    model = Model(inputs=[lstm_inputs, cnn_inputs], outputs=[output])
    model.compile(optimizer=optimizer, loss=loss, metrics=[metrix])
    return model

In [23]:
adam = keras.optimizers.Adam(lr=0.01)
classifier = classifier_3(optimizer=adam)
train_test_whole(classifier, 0, 'weights/weight-75overlap-model3.hdf5', spec_list, labe_list, epochs=50)
gc.collect()

c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  after removing the cwd from sys.path.
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  import sys


26.24495220184326
Train on 164164 samples, validate on 82075 samples
Epoch 1/50
164164/164164 [==============================] - 330s 2ms/step - loss: 0.0491 - acc: 0.9819 - val_loss: 0.0378 - val_acc: 0.9853
Epoch 2/50
164164/164164 [==============================] - 322s 2ms/step - loss: 0.0363 - acc: 0.9856 - val_loss: 0.0353 - val_acc: 0.9858
Epoch 3/50
  8992/164164 [>.............................] - ETA: 4:22 - loss: 0.0352 - acc: 0.9857

KeyboardInterrupt: 